In [1]:
from ananke.configurations.collection import HDF5StorageConfiguration
from ananke.models.collection import Collection
import os
from joblib import Parallel, delayed

In [2]:
path='LargeElectrical/1000.h5'
config=HDF5StorageConfiguration(data_path=path,read_only=False)
col=Collection(config)
with col:
    records=col.storage.get_records()
    hit_records=col.storage.get_hits().df['record_id'].drop_duplicates()

In [28]:
len(files)

883

In [27]:
path='LargeStarting/15records/'
files=[i for i in os.listdir(path) if len(i.split('.')[0])<7]

f=open('LargeStarting/CorrectRecords20R.txt','w')
for file in files:
    f.write(file+'\n')
f.close()

In [ ]:
config=HDF5StorageConfiguration(data_path=path+'83.h5',read_only=False)
col=Collection(config)
with col:
    records=col.storage.get_records().df
    hit_records=col.storage.get_hits().df['record_id'].drop_duplicates()

In [ ]:
records.shape

In [ ]:
hit_records

In [4]:
x=[]
for i ,file in enumerate(files):
    x.append(Check_data_integrity(file,path))
    break

In [5]:
x

[None]

In [ ]:
x=[i for i in x if i!=None]

In [12]:
def Check_data_integrity(file,path):
    try:
        config=HDF5StorageConfiguration(data_path=path+file,read_only=False)
        col=Collection(config)
        with col:
            records=col.storage.get_records().df
            hit_records=col.storage.get_hits().df['record_id'].drop_duplicates()
            #print(len(hit_records))
            if len(hit_records)!=15:
                return file,len(hit_records)
    except:
        return [file]

In [13]:
result=Parallel(n_jobs=8,verbose=20)(delayed(Check_data_integrity)(file,path) for file in files)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Batch computation too fast (0.0530s.) Setting batch_size=2.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:    1.0s
[Parallel(n_jobs=8)]: Done  15 tasks      | elapse

In [ ]:
result=[i for i in result if i !=None and type(i)!=list]
result

In [26]:
for file,_ in result:
    #print(file)
    try:
        os.remove(path+file)
    except:
        continue

In [ ]:
def get_statistics(file):
    try:
        stat_dict=dict()
        path='LargeCascades/20records/'
        config=HDF5StorageConfiguration(data_path=path+file,read_only=False)
        col=Collection(config)
        with col:
            stats=col.get_record_statistics().df
        stat_dict[file]=stats
        return stat_dict
    except:
        print(f'{file} had error')
        return file

In [ ]:
result=Parallel(n_jobs=8,verbose=20)(delayed(get_statistics)(file) for file in files)

In [ ]:
results=result

In [ ]:
error_files=[result for result in results if type(result)==str]
dict_df=[result for result in results if type(result)==dict]

In [ ]:
len(dict_df)

In [ ]:
merged_dict = {}
for d in dict_df:
    merged_dict.update(d)

print(len(merged_dict)) 

In [ ]:
import pandas as pd
from itertools import combinations
from joblib import Parallel, delayed


# Function to compare two DataFrames
def compare_keys(key_pair):
    key1, key2 = key_pair
    if merged_dict[key1].equals(merged_dict[key2]):
        return key_pair
    return None

# Generate all combinations of keys
key_combinations = list(combinations(merged_dict.keys(), 2))
len(key_combinations)

In [ ]:
len('/u/arego/project/Experimenting/data/LargeCascades/20records//9.h5')

In [ ]:
# Parallel processing
identical_combinations = Parallel(n_jobs=8, verbose=20)(
    delayed(compare_keys)(pair) for pair in key_combinations
)

# Filter out None values
identical_combinations = [pair for pair in identical_combinations if pair is not None]

print("Identical DataFrame combinations:", identical_combinations)

In [ ]:
def group_identical_keys(pairs):
    graph = defaultdict(set)
    
    # Build the graph where each key is connected to its identical counterparts
    for key1, key2 in pairs:
        graph[key1].add(key2)
        graph[key2].add(key1)

    # Use a set to track visited keys and group connected components
    visited = set()
    groups = []

    def dfs(node, group):
        if node not in visited:
            visited.add(node)
            group.append(node)
            for neighbor in graph[node]:
                dfs(neighbor, group)

    # Perform DFS for each unvisited node to form groups
    for node in graph:
        if node not in visited:
            group = []
            dfs(node, group)
            groups.append(group)

    return groups

# Get grouped identical keys
grouped_keys = group_identical_keys(identical_pairs)

# Output unique groups of keys
print("Grouped identical keys:")
for group in grouped_keys:
    print(sorted(group))  # Sorted for readability
